# Very dummy classifier

In [ ]:
%config IPCompleter.greedy=True

import yaml
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
#from os import path
import os
import os.path
import sys
from sqlalchemy import create_engine

pd.set_option("display.max_rows", 50)
pd.set_option('display.max_columns', None)

# Set the path
home_dir = os.getcwd()
credential_dir = os.path.join('/mnt/data/projects/el_salvador_mined_education', 'ana', 'db_credentials')

In [ ]:
def create_pgconn(credentials_yaml):
    with open(credentials_yaml) as f:
        configs = yaml.load(f)
    try: 
        conn = psycopg2.connect("dbname='{}' user='{}' host='{}' password='{}'".format(
            configs['database'],
            configs['user'],
            configs['host'],
            configs['password']))
    except: 
        print("Error connecting to db.")

    cur = conn.cursor()
    cur.execute("SET ROLE " + configs['role'])
    return conn
credentials_yaml = os.path.join(credential_dir, 'avaldivia_elsalvador.yaml') #example file on hitchikers repo
conn = create_pgconn(credentials_yaml)
def sql(query, conn=conn):
    return pd.read_sql(query, conn)

In [ ]:
# Read table from cleaned
df = sql("""
    SELECT *
    FROM cleaned.students_label
    WHERE year = 2016 AND label_1y is not null ;
""")

In [ ]:
# Create y_obs column
def f(row):
    if row['label_1y'] == 1:
        val = 1
    else:
        val = 0
    return val

df['y_obs'] = df.apply(f, axis=1) 

In [ ]:
# Set dummy classifier (all false in test set)
df['y_pred'] = 0

In [ ]:
df.head()

In [ ]:
# Compute some metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

print('accuracy:', accuracy_score(df.y_obs, df.y_pred))
# print('precision:', precision_score(df.y_obs, df.y_pred, average = 'macro'))
print('recall:', recall_score(df.y_obs, df.y_pred, average = 'macro'))
print('AUC:', roc_auc_score(df.y_obs, df.y_pred))
